In [32]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from src.activations import ActivationType, get_activation
from src.initialiaztion import get_initialization
from src.losses import LossType
import numpy as np

In [33]:
xavier_initialization = get_initialization("xavier")
he_initialization = get_initialization("he")

relu = get_activation("relu")[0]
sigmoid = get_activation("sigmoid")[0]
softmax = get_activation("softmax")

In [34]:
class NeuralNetwork:
  def __init__(self, 
               layer_dims: list[int], 
               activations: list[ActivationType], 
               loss_type=LossType, 
               optimizer_type : str = "gd",
               seed : int = 42):
    self._validate_inputs(layer_dims, activations, loss_type, optimizer_type)
    
    self.layer_dims = layer_dims
    self.activations = activations
    self.loss_type = loss_type
    self.optimizer_type = optimizer_type
    self.seed = seed
    self.params = {}
    
    self._initialize_parameters()
    
  def forward_pass(self, X: np.ndarray):
    # Validation
    if self.layer_dims[0] != X.shape[0]:
        raise ValueError(
            f"Input dimension mismatch. Expected {self.layer_dims[0]} features, "
            f"but got {X.shape[0]}."
        )

    L = len(self.layer_dims)
    A = X
    caches = [] 
    
    print(f"\n{'='*15} STARTING FORWARD PASS {'='*15}")
    print(f"Input Batch Shape: {X.shape} ({X.shape[1]} examples)")

    for i in range(1, L):
        act_name = self.activations[i - 1]
        act_obj = get_activation(act_name)
        
        if isinstance(act_obj, tuple):
            act_fnc = act_obj[0]
        else:
            act_fnc = act_obj

        W = self.params[f"W{i}"]
        b = self.params[f"b{i}"]
        A_prev = A
        
        Z = np.dot(W, A_prev) + b 
        A = act_fnc(Z)
        
        print(f"\n--- Layer {i} ({act_name.upper()}) ---")
        print(f"{'Input Matrix (A_prev)':<25} : {A_prev.shape}")
        print(f"{'Weight Matrix (W)':<25} : {W.shape}")
        print(f"{'Bias Vector (b)':<25} : {b.shape} (Broadcasts automatically)")
        print(f"{'-'*45}")
        print(f"{'Linear Step (Z = WA+b)':<25} : {Z.shape}")
        print(f"{'Activation (A = f(Z))':<25} : {A.shape}")
        
        caches.append((A_prev, Z))
    
    print(f"\n{'='*15} FORWARD PASS COMPLETE {'='*14}\n")
    return A, caches
    
  def _initialize_parameters(self):
      np.random.seed(self.seed)
      
      L = len(self.layer_dims)
      
      for i in range(1, L):
          D_o =  self.layer_dims[i]
          D_i = self.layer_dims[i - 1]
          act_fnc = self.activations[i - 1]
          
          if act_fnc == "relu":
              self.params[f"W{i}"] = he_initialization((D_o, D_i))
          if act_fnc == "sigmoid" or act_fnc == "softmax" or act_fnc == "linear":
              self.params[f"W{i}"] = xavier_initialization((D_o, D_i))
          
          self.params[f"b{i}"] = np.zeros((D_o, 1))
          
  def _validate_inputs(self, layer_dims, activations, loss_type, optimizer_type):
    """
    Private helper to validate all inputs before initialization.
    """
    if not isinstance(layer_dims, list):
        raise TypeError(f"layer_dims must be a list, got {type(layer_dims)}")
    
    if not all(isinstance(x, int) for x in layer_dims):
        raise TypeError("All elements in layer_dims must be integers!")

    if not isinstance(activations, list):
          raise TypeError(f"activations must be a list, got {type(activations)}")

    if len(layer_dims) < 2:
        raise ValueError("The length of layers must be at least 2 (Input -> Output)") 
    
    if min(layer_dims) < 1:
          raise ValueError("The number of neurons in every layer must be at least 1")
    
    if len(layer_dims) != len(activations) + 1:
          raise ValueError(
              f"Structure Error: You provided {len(layer_dims)} layers but {len(activations)} activations. "
              f"Expected {len(layer_dims) - 1} activations."
          )

    valid_activations = {"relu", "sigmoid", "softmax", "linear"}
    for act in activations:
        if act not in valid_activations:
            raise ValueError(f"Invalid activation '{act}'. Supported: {valid_activations}")

    valid_losses = {"mse", "bce", "cce"}
    if loss_type not in valid_losses:
        raise ValueError(f"Invalid loss_type '{loss_type}'. Supported: {valid_losses}")

In [35]:
nn = NeuralNetwork(
  activations=["relu", "relu", "relu", "sigmoid"],
  layer_dims=[3, 4, 2, 3, 2],
  loss_type="mse",
  optimizer_type="adam",
  seed=3
)

In [36]:
nn.params

{'W1': array([[ 1.46040903,  0.3564088 ,  0.07878985],
        [-1.52153542, -0.22648652, -0.28965949],
        [-0.06755814, -0.51194391, -0.03577739],
        [-0.38964689, -1.07276608,  0.72229115]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.]]),
 'W2': array([[ 0.62318596,  1.20885071,  0.03537913, -0.28615014],
        [-0.38562772, -1.0935246 ,  0.69463867, -0.77857239]]),
 'b2': array([[0.],
        [0.]]),
 'W3': array([[-1.18504653, -0.2056499 ],
        [ 1.48614836,  0.23671627],
        [-1.02378514, -0.7129932 ]]),
 'b3': array([[0.],
        [0.],
        [0.]]),
 'W4': array([[ 0.36098535, -0.09267243, -0.44388787],
        [-0.1328083 ,  0.43015844,  1.14090809]]),
 'b4': array([[0.],
        [0.]])}

In [37]:
X = np.random.normal(size=(3, 20))

y_pred = nn.forward_pass(X)[0]


=============== STARTING FORWARD PASS ===============
Input Batch Shape: (3, 20) (20 examples)

--- Layer 1 (RELU) ---
Input Matrix (A_prev)     : (3, 20)
Weight Matrix (W)         : (4, 3)
Bias Vector (b)           : (4, 1) (Broadcasts automatically)
---------------------------------------------
Linear Step (Z = WA+b)    : (4, 20)
Activation (A = f(Z))     : (4, 20)

--- Layer 2 (RELU) ---
Input Matrix (A_prev)     : (4, 20)
Weight Matrix (W)         : (2, 4)
Bias Vector (b)           : (2, 1) (Broadcasts automatically)
---------------------------------------------
Linear Step (Z = WA+b)    : (2, 20)
Activation (A = f(Z))     : (2, 20)

--- Layer 3 (RELU) ---
Input Matrix (A_prev)     : (2, 20)
Weight Matrix (W)         : (3, 2)
Bias Vector (b)           : (3, 1) (Broadcasts automatically)
---------------------------------------------
Linear Step (Z = WA+b)    : (3, 20)
Activation (A = f(Z))     : (3, 20)

--- Layer 4 (SIGMOID) ---
Input Matrix (A_prev)     : (3, 20)
Weight Matrix (W

In [39]:
y_pred[:, 3]

array([0.37234973, 0.91861426])